In [1]:
import numpy as np
from order_fulfillment_environment_notidentical_arrival_probs import OrderFulfillment
from gurobipy import Model, GRB
import gurobipy as gp

In [2]:
num_items = 5
n_max = 5
n_0 = 5
p_stock = 1
T = 10**4

instance = 1
alpha = 0.5 # if we want to change alpha, the demand of order arrivals needs to be changed

In [3]:
order_fulfillment = OrderFulfillment(num_items=num_items, n_max=n_max, n_0=n_0, p_stock=p_stock, T=T, CSL=0.5, 
                                    facilities_data="/Users/ayoubamil/Documents/Documents/GitHub/fulfillment-magician_AA_AM_YW/ORDER_FULFILLMENT/Data/fulfillment_centers_warmup_test.csv", 
                                    cities_data="/Users/ayoubamil/Documents/Documents/GitHub/fulfillment-magician_AA_AM_YW/ORDER_FULFILLMENT/Data/cities_warmup_test.csv", 
                                    prob_seed_value=instance, 
                                    order_seed_value=instance, 
                                    inv_seed_value=instance, 
                                    alpha=alpha)

In [4]:
I = order_fulfillment.num_items
K = len(order_fulfillment.facility_indices)
J = len(order_fulfillment.city_indices)
Q = len(order_fulfillment.df_orders)
print("I = ", I)
print("K = ", K)
print("J = ", J)
print("Q = ", Q)

I =  5
K =  2
J =  2
Q =  22


In [5]:
c_fixed = order_fulfillment.fixed_costs
c_unit = order_fulfillment.unit_costs

In [6]:
# [n][q][j]
# n is the size of the order
# q is the order type
# j is the location of the order
lambda_ = order_fulfillment.agg_adjusted_demand_distribution_by_type_by_location # same as not adjusted
S = order_fulfillment.safety_stock
order_types = order_fulfillment.order_types
print(order_types)

[(), (2,), (4,), (1,), (0,), (3,), (0, 1), (3, 4), (0, 3), (2, 3), (1, 3), (0, 1, 4), (0, 1, 3), (1, 2, 3), (0, 2, 3), (1, 3, 4), (0, 1, 3, 4), (0, 1, 2, 4), (1, 2, 3, 4), (0, 2, 3, 4), (0, 1, 2, 3), (0, 1, 2, 3, 4)]


In [7]:
# Reshape probabilities for the optimization
def reshape_probs(nested_distribution):
        """reshape a nested (3D) distribution"""
        
        reshape_arrival_prob = []
        
        for i in nested_distribution:
            for k in i:
                reshape_arrival_prob.append([j for j in k])
        return reshape_arrival_prob

lambda_ = reshape_probs(lambda_)

In [8]:
# Initialize model
model = Model()

# Define variables
u = model.addVars(Q, K + 1, I, J, vtype=GRB.CONTINUOUS, lb=0, name="u")
y = model.addVars(Q, K + 1, J, vtype=GRB.CONTINUOUS, lb=0, name="y")

# Objective function
model.setObjective(gp.quicksum(lambda_[q][j] * 
                    (c_fixed[k][j] * y[q, k, j] + c_unit[k][j] * gp.quicksum(u[q, k, i, j] for i in range(I) if i in order_types[q])) 
                    for q in range(Q) for j in range(J) for k in range(K + 1)), GRB.MINIMIZE)

# Add constraints
for q in range(Q):
    for j in range(J):
        for i in order_types[q]:
            model.addConstr(gp.quicksum(u[q, k, i, j] for k in range(K + 1)) == 1, f"order{q}_item{i}_city{j}_fulfilled")
            for k in range(K + 1):
                model.addConstr(y[q, k, j] >= u[q, k, i, j])

for i in range(I):
    for k in range(K): # no inventory constraints for K+1
        model.addConstr(gp.quicksum(gp.quicksum(lambda_[q][j] * u[q, k, i, j] for q in range(Q) if i in order_types[q]) for j in range(J)) <= S[k][i], f"InventoryConstraint_item{i}_facility{k}")

# Optimize
model.optimize()

# Print the number of decision variables and constraints
print(f'Number of decision variables: {model.NumVars}')
print(f'Number of constraints: {model.NumConstrs}')

if model.status == GRB.OPTIMAL:
    print('Optimal solution found.')
    for var in model.getVars():
        print(f'{var.varName} = {var.x}')
else:
    print('No optimal solution found.')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-12
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 450 rows, 792 columns and 1210 nonzeros
Model fingerprint: 0x335b5014
Coefficient statistics:
  Matrix range     [8e-02, 6e+02]
  Objective range  [5e-02, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 30 rows and 366 columns
Presolve time: 0.04s
Presolved: 420 rows, 426 columns, 1150 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0712774e+04   1.030000e+02   0.000000e+00      0s
     476    1.0166296e+05   0.000000e+00   0.000000e+00      0s

Solved in 476 iterations and 0.06 seconds (0.00 work units)
Optimal objective  1.016629638e+05
Number of decision variables: 792
Number of constraints: 450
Optimal solution found.
u[0,0,0,0] = 0.0
u[0,0,0,1] = 0.0
u[0

In [9]:
u_opt = model.getAttr('X', u)
XMatrices = np.zeros((Q, J, K, I)) # Rouding will be done only for the K facilities (excluding the dummy facility)

for q in range(Q):
    for j in range(J):
        for k in range(K):
            for i in range(I):
                XMatrices[q, j, k, i] = u_opt[q, k, i, j]

In [10]:
# Normalize the weights
lambda_normalized = lambda_ / np.sum(lambda_)
lambda_normalized_flatten = lambda_normalized.flatten()

# Sample from QJ based on weights
index_arrivals = np.random.choice(Q*J, size=10, p=lambda_normalized_flatten)

# Convert flat indices to QJ indices
q_indices = index_arrivals // J
j_indices = index_arrivals % J

# Pair Q and J indices
arrivals = [(q_indices[i], j_indices[i]) for i in range(len(index_arrivals))]

In [11]:
def dilate_round(x, q, j):
    
    K = x.shape[0]  # number of facilities
    n = x.shape[1]  # number of items
    
    fc_tried = np.zeros(n, dtype=int)
    
    opening_times = np.random.exponential(size=K)  # Exponential distribution
    
    for i in range(n):
        # Handle division by zero
        mask = x[:, i] == 0
        observed_opening = np.full(K, np.inf)  # Initialize all values to infinity
        observed_opening[~mask] = opening_times[~mask] / x[~mask, i]  # Perform division where x[:, i] is not zero
        
        fc_tried[i] = np.argmin(observed_opening)
        
    return fc_tried

In [12]:
def alg_run(rounding_function, arrivals, S, c_unit, c_fixed, order_types, K, XMatrices):
    
    tot_cost = 0
    rem_inv = S.copy()
    
    for t in range(len(arrivals)):
        
        q, j = arrivals[t] # q is the order index, j is the location of the order
        
        if order_types[q] == ():  # order is empty
            continue
        
        fc_tried = rounding_function(XMatrices[q, j], q, j)
        fc_used = [False] * (K + 1)
        
        for i in order_types[q]:
            k = fc_tried[i]
            if k <= K and rem_inv[k][i] > 0:
                rem_inv[k][i] -= 1
            else:
                k = K + 1
            fc_used[k] = True
            tot_cost += c_unit[k, j]
        
        for k in range(K + 1):
            if fc_used[k]:
                tot_cost += c_fixed[k, j]
    
    return tot_cost

In [13]:
alg_run(dilate_round, arrivals, S, c_unit, c_fixed, order_types, K, XMatrices)

93.82725319096494